## Recap
In the [previous article](https://diogenesanalytics.com/blog/2024/05/20/value-cost-ratio) the `VCR` (*value/cost* ratio) was introduced as a way to determine when potential *investment scenarios* are favorable (i.e. you can actually *gain* more utility by winning than what it *costs to play*). The example *scenario* was the [lottery](https://diogenesanalytics.com/blog/2024/05/19/optimal-options), and it was suggested that a $\text{vcr} > 1$ *could be favorable*. The key word there is **could**. In fact what we saw in the [previous article](https://diogenesanalytics.com/blog/2024/05/20/value-cost-ratio) suggested that you actually need to know if the *net expected utility* $E(n)$ will be increasing *or* decreasing to really know if it is favorable to play (see below for $E(n)$ equation):

$$
E(n) = (1 - p^n)U - nC
$$

If $E(n)$ was increasing from $n = 1$ onward, then it was possible to increase your *net expected utility* by playing repeated trials of the game (e.g. in the *lottery* scenario this meant buying *more tickets*, as each ticket represented a repeated play of the game). If $E(n)$ decreased, then the *highest* possible *net expected utility* could only be achieved at $n = 1$.

## The Derivative
To state this more plainly, if the first derivative of $E(n)$ is positive, then it should ultimately be *favorable*:

$$
\frac{dE(n)}{dn} > 0
$$

Else if the first derivative of $E(n)$ is negative, it should **not** be favorable:

$$
\frac{dE(n)}{dn} < 0
$$

And the actual first derivative of the $E(n)$ equation:

$$
\frac{dE(n)}{dn} = -p^n \ln(p) U - C
$$

And substituting for the first derivative equation and rearranging we have a *favorable* outcome:

$$
\frac{-p^n \ln(p) U}{C} > 1
$$

And a *not favorable* outcome:

$$
\frac{-p^n \ln(p) U}{C} < 1
$$

## Retry
Now let us attempt to *retry* the previous *example investment data* from the [previous article](https://diogenesanalytics.com/blog/2024/05/19/optimal-options) and see what happens:

In [ ]:
# libs
import random
from typing import Generator
from typing import Tuple

# seed rng
random.seed(42)

# select random number from partitions
def gen_partition_ranges(start: int, end: int, partitions: int) -> Generator[Tuple[float, float], None, None]:
    """Generate sub-ranges from partitions of a given range."""    
    # calculate partition sizes with a shifted distribution
    partition_sizes = [((i + 1) ** 2) for i in range(partitions)]
    total_partition_size = sum(partition_sizes)

    # adjust partition sizes to fit the range
    normal_partition_sizes = [size / total_partition_size * (end - start) for size in partition_sizes]

    # begin iterating over partitions
    for partition, partition_size in enumerate(normal_partition_sizes):
        # calculate partition bounds
        partition_start = start + partition * partition_size
        partition_end = partition_start + partition_size

        # generate i-th partition range
        yield partition_start, partition_end

def gen_random_partition(start: int, end: int, partitions: int) -> Generator[Tuple[float, float], None, None]:
    """Generate random numbers from partitions of a given range."""
    # loop over partition
    for start, end in gen_partition_ranges(start, end, partitions):
        # get random float
        yield random.uniform(start, end)

# func for U/C/p pairs
def gen_investments(partitions: int) -> Generator[Tuple[float, float, float], None, None]:
    """Generate pairs of U/C/p values."""    
    # get random partition samples for U/C
    uc_random_samples = list(gen_random_partition(1, 1e7, partitions))
    
    # loop until U > C
    while True:        
        # generate U and C as positive non-zero integers
        U = int(random.choice(uc_random_samples))
        C = int(random.choice(uc_random_samples))
        
        # termination condition
        if U > C:
            break

    # get random partition samples for p
    p_random_samples = list(gen_random_partition(1e-6, 1 - 1e-6, partitions))
    
    # generate 0 < p < 1
    p = random.choice(p_random_samples)

    # check if greater than 1
    p = p if p < 1 else 1 / p

    # get U/C/p pair
    return U, C, p

In [ ]:
# get lib for log
import math

# define functions
def expected_utility(U: float, C: float, p: float, n: int) -> float:
    """Calculates the expected utility."""
    return ((1 - p**n) * U) - (n * C)
    
def derivative_net_expect_utility(U: float, C: float, p: float, n: int) -> float:
    """Calculates the derivative of the expected utility."""
    return (-U * (p**n) * math.log(p)) - C
    
def new_vcr(U: float, C: float, p: float, n: int) -> float:
    """Calculates the new value/cost ratio."""
    return (-(p**n) * math.log(p) * U) / C

In [ ]:
# get libs
import pandas as pd

# generate example investments
example_investments = [gen_investments(1000) for _ in range(20)]

# create table
example_investments_table = pd.DataFrame(example_investments, columns=["U", "C", "p"])

# set index name
example_investments_table.index.name = "id"

# add new column for E(n) (evaluated at n = 1)
example_investments_table["E(n)"] = example_investments_table.apply(
    lambda row: expected_utility(row["U"], row["C"], row["p"], 1),
    axis=1
)

# add new column for dE(n) (evaluated at n = 1)
example_investments_table["dE/dn"] = example_investments_table.apply(
    lambda row: derivative_net_expect_utility(row["U"], row["C"], row["p"], 1),
    axis=1
)

# add new colum for dVCR (evaluated at n = 1)
example_investments_table["vcr_redux"] = example_investments_table.apply(
    lambda row: new_vcr(row["U"], row["C"], row["p"], 1),
    axis=1
)

In [ ]:
# get plotting libs
import matplotlib.pyplot as plt

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# pointer to long example investment table name
eit = example_investments_table

# create dictionary for favorability groups
favorable_groups = {
    "Not Favorable": eit["vcr_redux"] < 1,
    "Favorable": eit["vcr_redux"] > 1,
}

# loop through favorable groups
for fig_num, (key, mask) in enumerate(favorable_groups.items()):
    # get sub data frame
    sub_data_frame = eit[mask]
    
    # loop over rows
    for idx, row in sub_data_frame.iterrows():
        # get values
        U = row["U"]
        C = row["C"]
        p = row["p"]
        
        # generate x/y pairs
        x_values, y_values = zip(*((n, expected_utility(U, C, p, n)) for n in range(1, 11)))
    
        # create the line plot
        plt.plot(x_values, y_values, label=idx)

    # add titles and labels
    plt.xlabel("Number of Options (n)")
    plt.ylabel("Expected Utility (E)")
    
    # set title
    plt.suptitle(
        f"Figure {fig_num + 1}. Optimal Number for {key!r} Investments", y=0.0001, fontsize=10
    )

    # now show
    plt.legend(title="id#")
    plt.show()

The two figures above show the results of using the *new VCR* equation based on $\frac{dE(n)}{dn}$ (shown below):

$$
\frac{-p^n \ln(p) U}{C}
$$

This *new VCR equation* works extremely well to indicate which *investments* are favorable, not just in terms of having a *positive net expected utility* but also having an *optimal options* value $n$. In truth one could simply just use the $\frac{dE(n)}{dn}$ equation evaluated at $n = 1$ to make the same judgement (this can be seen in the table below, where all values *E(n)*, *dE/dn*, and *vcr_redux* are evaluated at $n = 1$).

In [ ]:
# print table
print(example_investments_table.to_string())

Notice how in the above table, everywhere that `dE/dn` $< 0$ the `vcr_redux` $< 1$? Again, you can simply use the $\frac{dE(n)}{dn}$ equation evaluated at $n = 1$ to get the same results as the *new VCR equation* (`vcr_redux`). Also notice that everywhere that $E(n)$ (evaluated at $n = 1$) is *negative* in the above table, the corresponding $\frac{dE(n)}{dn}$ is also negative.

## Moral
As it turns out, a little *calculus* can tell us *a lot* about what a *function* is doing. In this specific application, it allows us to determine when a specific set of *U*, *C*, and *p* values not only yield a *positive net expected utility* (i.e. $E(n) > 0$) but whether this particular *investment* has a maximum value somewhere in the range $n > 0$ (meaning that there is possibility for purchasing $n >=1$ *options* to increase $E(n)$.